# 고등 수학 학습 단원 기반 문제 제공 및 분석 프로그램

이 프로젝트는 AI-Hub에서 제공하는 고등 수학 라벨링 데이터를 활용하여 단원을 입력하면 해당 단원에 맞는 수학 문제를 자동으로 생성하는 프로그램입니다. 이 데이터는 문제와 모범 답안이 별도의 JSON 파일로 제공되며, 각 파일의 `id`를 기준으로 문제와 답을 매칭하여 학습 데이터를 구성합니다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 출처

- **출처**: AI-Hub
- **데이터명**: 수학 과목 문제 생성 데이터
- **설명**: 고등학교 수학 문제와 모범 답안을 포함하며, 각 문제는 문제의 난이도, 단원명, 유형 등의 정보가 포함된 JSON 파일로 제공됩니다. 답안 또한 별도의 JSON 파일로 저장되어 있으며, `id`를 기준으로 문제와 결합할 수 있습니다.


In [ ]:
! unzip "/content/drive/MyDrive/111.수학 과목 문제생성 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL_1.문제_고등학교_공통수학.zip" -d './data'

In [ ]:
! unzip "/content/drive/MyDrive/111.수학 과목 문제생성 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL_2.모범답안_고등학교_공통수학.zip" -d './answer'

# 1. **데이터 로드 및 전처리**:
   - 문제와 답안을 각각 로드하고 `id`를 기준으로 결합하여 문제-답안 쌍을 구성
   - 각 문제의 단원명, 난이도, 문제 텍스트, 모범 답안을 포함하는 학습 데이터를 준비

In [27]:
import json
import os

# JSON 파일 경로
question_folder_path = '/content/data'
answer_folder_path = '/content/answer'

# JSON 파일 로드 함수
def load_json_files(folder_path):
    data = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                # 각 파일의 id를 키로 사용하여 저장
                data[json_data['id']] = json_data
    return data

# 문제와 답 파일 로드
questions_data = load_json_files(question_folder_path)
answers_data = load_json_files(answer_folder_path)

# 문제와 답을 결합하여 전처리
combined_data = []
for question_id, question_item in questions_data.items():
    if question_id in answers_data:
        question_info = question_item.get('question_info', [{}])[0]
        question_text = question_item.get('OCR_info', [{}])[0].get('question_text', '')
        question_difficulty = question_info.get('question_difficulty', None)

        answer_text = answers_data[question_id].get('answer_info', [{}])[0].get('answer_text', '')

        if question_info and question_text and answer_text:
            combined_data.append({
                'topic_name': question_info.get('question_topic_name', ''),
                'question_text': question_text,
                'difficulty': question_difficulty,
                'answer_text': answer_text
            })



In [23]:
!sudo apt-get install texlive-latex-base dvipng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lmodern fonts-noto-mono fonts-urw-base35
  ghostscript libfontenc1 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 libkpathsea6 libptexenc1 libsynctex2 libteckit0 libtexlua53
  libtexluajit2 libwoff1 libzzip-0-13 lmodern poppler-data t1utils tex-common
  texlive-base texlive-binaries xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum debhelper perl-tk xpdf | pdf-viewer xzdec
  texlive-latex-base-doc
The following NEW packages will be installed:
  dvipng dvisvgm fonts-droid-fallback fonts-lmodern fonts-noto-mono
  fonts-urw-base35 ghostscript libfontenc1 libgs

In [29]:
# 예제 데이터 출력
for item in combined_data[:5]:  # 상위 5개만 출력
    print(f"단원명: {item['topic_name']}")
    print(f"난이도: {item['difficulty']}")
    print("문제:")
    display(f"{item['question_text']}")
    print("모범 답안:")
    display(f"{item['answer_text']}")
    print()


단원명: 집합의 연산 법칙
난이도: 3
문제:


'세 집합 $A,  B,  C$에 대하여 $A\\cap B=\\lbrace2, 3, 4\\rbrace,  A\\cap C=\\lbrace3, 5\\rbrace$일 때, $A\\cap(B\\cup C)$를 구하시오.  $A\\cap(B\\cup C)=$'

모범 답안:


'$A\\cap(B\\cup C)$ $= (A\\cap B)\\cup(A\\cap C)$ $= \\lbrace2, 3, 4\\rbrace\\cup\\lbrace3, 5\\rbrace$ $= \\lbrace2, 3, 4, 5\\rbrace$'


단원명: 식의 변형을 이용한 다항식의 인수분해
난이도: 2
문제:


...

"명제의 부정은 '모든 실수 $x$에 대하여 $x^2-x+3>0$이다.' $x^2-x+3=(x-\\frac{1}{2})^2+\\frac{11}{4}>0$이므로 명제의 부정은 참이다."


단원명: 유한집합의 원소의 개수
난이도: 3
문제:


'다음 중 옳지 않은 것은? ① $n(\\lbrace1, 2, 3\\rbrace)-n(\\lbrace5, 6, 7\\rbrace)$ ② $A=\\lbrace0\\rbrace$이면 $n(A)=1$이다. ③ $n(A)=0$이면 $A=\\lbrace\\varnothing\\rbrace$이다. ④ $n(\\lbrace\\varnothing\\rbrace)-n(\\varnothing)=1$ ⑤ $n(\\lbrace20\\rbrace)-n(\\lbrace15\\rbrace)=0$'

모범 답안:


'① $n(\\lbrace1, 2, 3\\rbrace)=n(\\lbrace5, 6, 7\\rbrace)=3$ ② $A=\\lbrace0\\rbrace$이면 $n(A) =1$이다. ③ $n(A)=0$이면 $A=\\varnothing$이다. ④ $n(\\lbrace\\varnothing\\rbrace)-n(\\varnothing) =1-0 =1$ ⑤ $n(\\lbrace20\\rbrace)-n(\\lbrace15\\rbrace) =1-1 =0$'

In [31]:
# 단원 리스트 추출 함수
def extract_unique_topics(data):
    topics = set()  # 고유한 단원명을 저장하기 위해 집합 사용
    for item in data:
        topic_name = item.get('topic_name', '')
        if topic_name:
            topics.add(topic_name)
    return sorted(list(topics))  # 정렬된 리스트로 반환

# JSON 데이터에서 단원 리스트 추출
topic_list = extract_unique_topics(combined_data)

# 단원 리스트 출력
print("선택할 수 있는 단원 리스트:")
for topic in topic_list:
    print(f"- {topic}")

선택할 수 있는 단원 리스트:
-  x, y를 바꾸어도 식이 변하지 않는 연립이차방정식
-  x³=-1의 허근의 성질
- (분자의 차수)≥(분모의 차수)인 유리식
- A < B < C 꼴의 부등식의 풀이
- A⊂X⊂B를 만족시키는 집합 X
- f(x, y)=0의 평행이동과 대칭이동
- f=f^(-1)인 함수
- f○g=g○f인 경우
- f○g=h를 만족시키는 함수 f 또는 g 구하기
- f○g에 대한 조건이 주어진 경우
- x±1/x을 포함한 곱셈 공식의 변형
- x절편, y절편이 주어진 직선의 방정식
- x축 또는 y축에 접하는 원의 방정식
- 거짓인 명제의 반례
- 계수가 문자인 이차방정식의 근의 판별
- 곱셈 공식
- 곱셈 공식을 이용한 다항식의 전개
- 곱셈 공식을 이용한 수의 계산
- 곱셈 공식의 도형에의 활용
- 공통부분이 있는 다항식의 인수분해
- 공통부분이 있는 다항식의 전개
- 귀류법
- 그래프를 이용한 부등식의 풀이
- 근과 계수의 관계
- 근과 계수의 관계를 이용하여 식의 값 구하기
- 근이 주어진 삼차방정식
- 기호 ∈, ⊂의 사용
- 나머지정리
- 나머지정리를 이용한 수의 나눗셈
- 다각형의 개수
- 다항식의 나눗셈과 항등식
- 다항식의 덧셈과 뺄셈
- 다항식의 전개
- 대우를 이용한 명제의 증명
- 대칭이동
- 대칭이동을 이용한 거리의 최솟값
- 도형에서의 활용
- 도형의 대칭이동
- 도형의 평행이동
- 두 이차방정식으로 이루어진 연립이차방정식
- 두 점 사이의 거리
- 두 점 사이의 거리의 활용
- 두 점을 지름의 양 끝 점으로 하는 원의 방정식
- 두 점이 주어진 직선의 방정식
- 두 직선의 위치 관계
- 두 집합의 포함 관계
- 드모르간의 법칙
- 명제
- 명제가 참이 되도록 하는 상수 구하기
- 명제의 대우를 이용하여 상수 구하기
- 명제의 역, 대우의 참, 거짓
- 명제의 참, 거짓
- 명제의 참, 거짓과 진리집합
- 모든'이나 '어떤'이 있는 명제
- 몫과 나머지
- 무리식의 값 구하기
- 무리식의 값이 실수가 되기 위한 조

# GPT-2 기반 수학 문제 및 답 생성 모델

이 프로젝트는 `GPT-2` 모델을 활용하여 단원명과 난이도를 입력받아 수학 문제와 답을 생성하는 모델을 학습시킵니다. 학습 데이터는 고등학교 수학 문제와 답을 포함하며, Python과 Hugging Face의 `transformers` 라이브러리를 사용해 모델을 구현했습니다.

### 1. 모델 및 토크나이저 로드
`GPT-2` 모델과 토크나이저를 Hugging Face 라이브러리를 통해 불러옵니다.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

### 2. 학습 데이터 준비
JSON 데이터에서 문제와 답을 추출하고 학습에 적합한 형식으로 변환하여 저장


In [ ]:
def format_data_for_training(data):
    formatted_data = []
    for item in data:
        input_text = f"단원: {item['topic_name']}\n난이도: {item['difficulty']}\n문제와 답을 생성:\n"
        output_text = f"문제: {item['question_text']}\n답: {item['answer_text']}\n"
        formatted_data.append(input_text + output_text)
    return formatted_data

training_data = format_data_for_training(combined_data)

# 학습 데이터 파일로 저장
with open("training_data.txt", "w", encoding="utf-8") as f:
    for line in training_data:
        f.write(line + "\n")

### 3. 학습 데이터셋 생성
저장된 텍스트 데이터를 `transformers`의 `TextDataset` 클래스를 사용하여 학습 데이터셋으로 변환합니다.

In [ ]:
from transformers import TextDataset

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="training_data.txt",
    block_size=128
)

### 4. 데이터 컬레이터 설정
데이터 배치를 구성하기 위한 `DataCollatorForLanguageModeling` 설정입니다.

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

### 5. 학습 설정
모델 학습을 위한 파라미터를 설정



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    prediction_loss_only=True
)

### 6. 트레이너 설정 및 모델 학습
`Trainer` 클래스를 사용해 모델 학습을 설정하고 학습을 시작



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()


### 7. 모델 저장
학습된 모델을 저장

In [ ]:
trainer.save_model("./trained_model")

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 모델 및 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("./trained_model")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 단원명과 난이도를 입력받아 문제와 답 생성
input_text = "단원: 집합의 연산 법칙\n난이도: 3\n문제와 답을 생성:\n"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=150, num_return_sequences=1, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)